In [ ]:
import pyemu
import os, shutil
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import geostat_helpers as gh
import pandas as pd
from scipy.stats.mstats import normaltest
from scipy.stats import norm
import scipy.stats as sps

# Geostatistics 
### Some definitions from Geoff Bohling http://people.ku.edu/~gbohling/cpe940/Variograms.pdf
> ## “Geostatistics: study of phenomena that vary in space and/or time”
(Deutsch, 2002)


> ## “Geostatistics can be regarded as a collection of numerical techniques that deal with the characterization of spatial attributes, employing primarily random models in a manner similar to the way in which time series analysis characterizes temporal data.”
(Olea, 1999)


> ## “Geostatistics offers a way of describing the spatial continuity of natural phenomena and provides adaptations of classical regression techniques to take advantage of this continuity.” 
(Isaaks and Srivastava, 1989)


> ## Geostatistics deals with spatially _autocorrelated_ data.

> ## Autocorrelation: correlation between elements of a series and others from the same series separated from them by a given interval. 
(Oxford American Dictionary)

# Main Concepts


## 1. Variogram modeling -- a way to characterize spatial correlation
## 2. Kriging -- a best linear unbiased estimate (BLUE) for interpolation with minimum variance. There are several flavors - we will focus on Ordinary Kriging
## 3. Stochastic Simulation -- http://petrowiki.org/Geostatistical_conditional_simulation
## 4. Beyond this multi-Gaussian approach focused on the relationships among pairs of points, there is _multiple point geostatistics_ as well using training images and more complex shapes

# These concepts each build on each other. We will briefly touch on the first two

## let's cook up a quick random field and explore the spatial structure

In [ ]:
X,Y,Z,v,gs,sample_df = gh.data_cooker()

## Pretend (key word!) that this is a hydraulic conductivity field

## Any _autocorrelation_ here?

In [ ]:
gh.grid_plot(X,Y,Z)

## Of course, we would typically only know the values at a few points 
## (and probably not perfectly)
## N.B. --> The default number of samples used here is 50

In [ ]:
gh.field_scatterplot(sample_df.x,sample_df.y,sample_df.z)

====================================

# Geostatistics is based on a couple main assumptions:
   ## 1. The values are second order stationary (the mean and variance are relatively constant) 
   ## 2. The values are multi-Gaussian (e.g. normally distributed)

# Are the raw data normally distributed?

### Spoiler alert - this field was generated using a variogram!

In [ ]:
plt.hist(Z.ravel(), bins=50, density=True)
x=np.linspace(min(Z.ravel()),max(Z.ravel()),100)
plt.plot(x,sps.norm.pdf(x, np.mean(Z),np.std(Z)))

In [ ]:
normaltest(Z.ravel())

# How about our subsample?

In [ ]:
plt.hist(sample_df.z, bins=50, density=True)
x=np.linspace(min(Z.ravel()),max(Z.ravel()),100)
plt.plot(x,sps.norm.pdf(x, np.mean(sample_df.z),np.std(sample_df.z)))

In [ ]:
normaltest(sample_df.z)

# That was a pretty small sample....
## Explore a bigger sample

In [ ]:
n_sample_pts=1000
xd = np.random.uniform(0, 1000, n_sample_pts)
yd = np.random.uniform(0, 1000, n_sample_pts)
z = gh.sample_from_grid(X,Y,Z,xd,yd)
plt.hist(z, bins=50, density=True)
x=np.linspace(min(Z.ravel()),max(Z.ravel()),100)
plt.plot(x,sps.norm.pdf(x, np.mean(z),np.std(z)))
normaltest(z)

## Purity is commendable, but we are going to violate some of these assumptions for sure

====================================

# At the heart of geostatistics is some kind of model expressing the variability of properties in a field

## This is a "variogram" and we can explore it based on the following empirical formula
## $\hat{\gamma}\left(h\right)=\frac{1}{2\left(h\right)}\left(z\left(x_1\right)-z\left(x_2\right)\right)^2$
## where $x_1$ and $x_2$ are the locations of two $z$ data points separated by distance $h$

## If we plot these up we get something called a cloud plot showing $\hat\gamma$ for all pairs in the dataset

In [ ]:
h,gam,ax=gh.plot_empirical_variogram(sample_df.x,sample_df.y,sample_df.z,0)

## This is pretty messy, so typically it is evaluated in bins, and usually only over half the total possible distance

In [ ]:
h,gam,ax=gh.plot_empirical_variogram(sample_df.x,sample_df.y,sample_df.z,50)

## Also note that this was assuming perfect observations. What if there was ~10% noise?

In [ ]:
h,gam,ax=gh.plot_empirical_variogram(sample_df.x,sample_df.y,sample_df.z_noisy,30)

## Geostatistics is making the assumption that you can model the variability of this field using a variogram. The variogram is closely related to covariance. 

## We take advantage of a few assumptions to come up with a few functional forms that should characterize this behavior

## `pyemu` supports three variogram models

## This follows the _GSLIB_ terminology

 ## 1. *Spherical*  
### $\gamma\left(h\right)=c\times\left[1.5\frac{h}{a}-0.5\frac{h}{a}^3\right]$ if $h<a$
### $\gamma\left(h\right)=c$ if $h \ge a$  
     
 ## 2. *Exponential*  
### $\gamma\left(h\right)=c\times\left[1-\exp\left(-\frac{h}{a}\right)\right]$  
     
 ## 3. *Gaussian*  
### $\gamma\left(h\right)=c\times\left[1-\exp\left(-\frac{h^2}{a^2}\right)\right]$  

     

### $h$ is the separation distance, and $a$ is the range. `contribution` is the variogram value at which the variogram levels off. Also called the `sill`, this value is the maximum variability between points.

### The sill is reached at about $a$ for the *Spherical* model, $2a$ for the *Gaussian*, and $3a$ for the *Exponential*

# What do these look like?

## for a consistent set of parameters:
## a=500, c=10
## We can use `pyemu` to setup a geostatistical model

In [ ]:
a=500
c=10

## Set up a variogram object and, from that, build a geostatistical structure

### _Spherical_

In [ ]:
v = pyemu.geostats.SphVario(contribution=c, a=a)
gs = pyemu.geostats.GeoStruct(variograms=v)

In [ ]:
gs.plot()
plt.plot([v.a,v.a],[0,v.contribution],'r')
plt.grid()

In [ ]:
Q= gs.covariance_matrix(X.ravel(), Y.ravel(), names=[str(i) for i in range(len(Y.ravel()))])
plt.figure(figsize=(6,6))
plt.imshow(Q.x)
plt.colorbar()

### _Exponential_

In [ ]:
v = pyemu.geostats.ExpVario(contribution=c, a=a)
gs = pyemu.geostats.GeoStruct(variograms=v)
gs.plot()
plt.plot([v.a,v.a],[0,v.contribution],'r')
plt.plot([3*v.a,3*v.a],[0,v.contribution],'r:')
plt.grid()

In [ ]:
Q= gs.covariance_matrix(X.ravel(), Y.ravel(), names=[str(i) for i in range(len(Y.ravel()))])
plt.figure(figsize=(6,6))
plt.imshow(Q.x)
plt.colorbar()

### _Gaussian_

In [ ]:
v = pyemu.geostats.GauVario(contribution=c, a=a)
gs = pyemu.geostats.GeoStruct(variograms=v)
gs.plot()
plt.plot([v.a,v.a],[0,v.contribution],'r')
plt.plot([7/4*v.a,7/4*v.a],[0,v.contribution],'r:')
plt.grid()

In [ ]:
Q= gs.covariance_matrix(X.ravel(), Y.ravel(), names=[str(i) for i in range(len(Y.ravel()))])
plt.figure(figsize=(6,6))
plt.imshow(Q.x)
plt.colorbar()

# If we fit an appropriate model ($\gamma$) to the empirical variogram ($\hat\gamma$), we can use that structure for interpolation from sparse data

In [ ]:
h,gam,ax=gh.plot_empirical_variogram(sample_df.x,sample_df.y,sample_df.z,50)
new_c=10.0
new_a=300.0

v_fit = pyemu.geostats.ExpVario(contribution=new_c,a=new_a)
gs_fit = pyemu.geostats.GeoStruct(variograms=v_fit)
gs_fit.plot(ax=ax)


In [ ]:
Q = gs_fit.covariance_matrix(X.ravel(), Y.ravel(), names=[str(i) for i in range(len(Y.ravel()))])

In [ ]:
plt.figure(figsize=(6,6))
plt.imshow(Q.x)
plt.colorbar(shrink=.75)


# We can perform Kriging to interpolate using this variogram and our "data"

## First make an Ordinary Kriging object

In [ ]:
k = pyemu.geostats.OrdinaryKrige(gs_fit,sample_df)

In [ ]:
sample_df.head()

## Next we need to calculate factors (we only do this once - takes a few seconds)

In [ ]:
kfactors = k.calc_factors(X.ravel(),Y.ravel(), num_threads=10)

In [ ]:
kfactors = k.calc_factors(X.ravel(),Y.ravel())

## It's easiest to think of these factors as weights on surrounding point to calculate a weighted average of the surrounding values. The weight is a function of the distance - farther points have smaller weights

In [ ]:
kfactors.head()

In [ ]:
Z_interp = gh.geostat_interpolate(X,Y,k.interp_data, sample_df)

In [ ]:
vlims = [Z.min(),Z.max()]
ax=gh.grid_plot(X,Y,Z_interp,vlims=vlims, title='reconstruction')
ax.plot(sample_df.x,sample_df.y, 'ko')

In [ ]:
gh.grid_plot(X,Y,Z,vlims=vlims,title='truth')

In [ ]:
ax=gh.grid_plot(X,Y,kfactors.err_var.values.reshape(X.shape), title='Variance of Estimate')
ax.plot(sample_df.x,sample_df.y, 'ko')

In [ ]:
ax=gh.grid_plot(X,Y,np.abs(Z-Z_interp), title='Actual Differences')
ax.plot(sample_df.x,sample_df.y, 'yo')

# What if our data were noisy?

In [ ]:
h,gam,ax=gh.plot_empirical_variogram(sample_df.x,sample_df.y,sample_df.z_noisy,30)
new_c=15.0
new_a=600.0

# select which kind of variogram here because in reality we don't know, right?
v_fit = pyemu.geostats.ExpVario(contribution=new_c,a=new_a)
gs_fit = pyemu.geostats.GeoStruct(variograms=v_fit, nugget=0)
gs_fit.plot(ax=ax)

In [ ]:
Q = gs_fit.covariance_matrix(X.ravel(), Y.ravel(), names=[str(i) for i in range(len(Y.ravel()))])
plt.figure(figsize=(6,6))
plt.imshow(Q.x)
plt.colorbar(shrink=.75);

## Again make the Kriging Object and the factors and interpolate

In [ ]:
k = pyemu.geostats.OrdinaryKrige(gs_fit,sample_df)
kfactors = k.calc_factors(X.ravel(),Y.ravel())
Z_interp = gh.geostat_interpolate(X,Y,k.interp_data, sample_df)

In [ ]:
ax=gh.grid_plot(X,Y,Z_interp,vlims=vlims, title='reconstruction')
ax.plot(sample_df.x,sample_df.y, 'ko')

In [ ]:
gh.grid_plot(X,Y,Z,vlims=vlims,title='truth')

In [ ]:
ax=gh.grid_plot(X,Y,kfactors.err_var.values.reshape(X.shape), title='Variance of Estimate')
ax.plot(sample_df.x,sample_df.y, 'ko')

In [ ]:
ax=gh.grid_plot(X,Y,np.abs(Z-Z_interp), title='Actual Differences')
ax.plot(sample_df.x,sample_df.y, 'yo')

### Spectral simulation

Because pyemu is pure python (and because the developers are lazy), it only implments spectral simulation for grid-scale field generation.  For regular grids without anisotropy and without conditioning data ("known" property values), it is identical to sequential gaussian sim

In [ ]:
ev = pyemu.geostats.ExpVario(1.0,1)
gs = pyemu.geostats.GeoStruct(variograms=ev)
ss = pyemu.geostats.SpecSim2d(np.ones(100),np.ones(100),gs)
plt.imshow(ss.draw_arrays()[0])

In [ ]:
ev = pyemu.geostats.ExpVario(1.0,5)
gs = pyemu.geostats.GeoStruct(variograms=ev)
ss = pyemu.geostats.SpecSim2d(np.ones(100),np.ones(100),gs)
plt.imshow(ss.draw_arrays()[0])

In [ ]:
ev = pyemu.geostats.ExpVario(1.0,500)
gs = pyemu.geostats.GeoStruct(variograms=ev)
ss = pyemu.geostats.SpecSim2d(np.ones(100),np.ones(100),gs)
plt.imshow(ss.draw_arrays()[0])

# Further resources and information

   # These concepts are used for pilot point interpolation in PEST
   ## a. In the GW utilities in PEST (http://www.pesthomepage.org/Groundwater_Utilities.php) 
   ## b. The main tools are also available in `pyemu` -- we'll use that in the class

# A nice but outdated set of references is at: http://people.ku.edu/~gbohling/geostats/

# The Stanford Geostatistical Modeling Software (SGeMS: http://sgems.sourceforge.net/) is a nice GUI for geostatistical modeling, but it's not being maintained anymore.

# `R` has a package: http://rgeostats.free.fr/

# Some packages are under development in `python` but not very far along. 

# `QGIS` and `ArcGIS` have some geostatistical modeling capabilities as well

# Implementing Pilot Points in a `MODFLOW` model with `pyemu`

In [ ]:
import flopy

In [ ]:
t_d = "temp_history"
m = flopy.modflow.Modflow.load("freyberg.nam",model_ws=t_d,check=False,forgive=False)

In [ ]:
m.get_package_list()

## Set up zones for where pilot points will be interpolated

We can have pilot point networks in multiple zones. In this case, we will make a simple zone file using `IBOUND` such that all active cells are in the same interpolation zone.

In [ ]:
m.bas6.ibound.plot()

### We don't want pilot points or care about HK values in inactive cells, but we do need values in constant heads

We are going to use a `pyemu` helper function to setup pilot points are cell centers for active cells in layer 2.

In [ ]:
working_dir = 'pp_tmp'
if os.path.exists(working_dir):
    shutil.rmtree(working_dir)
os.mkdir(working_dir)

In [ ]:
# we want hk pilot points in the top layer...
prefix_dict = {1:["hk"]}  # note, layer 1 is the neew layer 2 (e.g. zero-based)
df_pp = pyemu.utils.setup_pilotpoints_grid(ml=m,prefix_dict=prefix_dict,
                                              pp_dir=working_dir,
                                              tpl_dir=working_dir,
                                              every_n_cell=3)
pp_file = os.path.join(working_dir,"hkpp.dat")
assert os.path.exists(pp_file)

In [ ]:
df_pp

## Need to create Kriging factors and regularization inputs
Following the guidelines in _Approaches to Highly Parameterized Inversion: Pilot-Point Theory, Guidelines, and Research Directions_ https://pubs.usgs.gov/sir/2010/5168/

### First, let's create ``variogram`` and ``GeoStruct`` objects.  

These describe how HK varies spatailly, remember?

In [ ]:
v = pyemu.geostats.ExpVario(contribution=1.0,a=2500)
gs = pyemu.geostats.GeoStruct(variograms=v,nugget=0.0)
ax = gs.plot()
ax.grid()
ax.set_ylim(0,2.0)

Now, let's get an ``OrdinaryKrige`` object, which needs the ``GeoStruct`` as well as the x, y, and name of the pilot point locations (which happens to be in that really cool ``df_pp`` instance)

In [ ]:
ok = pyemu.geostats.OrdinaryKrige(gs,df_pp)

Once the ``OrdinaryKrige`` is created, we need to calculate the geostatistical interpolation factors for each model cell.  We do this with the ``.calc_factors_grid()`` method: it needs to know about the model's spatial orientation and also accepts some optional arguments:

In [ ]:
df = ok.calc_factors_grid(m.sr,var_filename=os.path.join(working_dir,'kriging.var'),
                          minpts_interp=1,maxpts_interp=10)

One of the really cool things about geostatistics is that it gives you both the interpolation (factors), but also gives you the uncertainty in the areas between control (pilot) points.  Above, we wrote this uncertainty information to an array that has the same rows and cols as the model grid - this array is very useful for understanding the function of the variogram.

In [ ]:
# plot the kriging variance
arr_var = np.loadtxt(os.path.join(working_dir,'kriging.var'))
ax = plt.subplot(111,aspect="equal")
p = ax.imshow(arr_var,extent=m.sr.get_extent(),alpha=0.25)
plt.colorbar(p)
ax.scatter(df_pp.x,df_pp.y,marker='.',s=4,color='r')
ax.set_title('Kriging Variance')

We see that at the pilot point locations (red dots), the uncertainty in the geostats is minimal...as expected. The call to ``.calc_factors_grid()`` also returns a ``DataFrame`` which has useful info - lets look:

In [ ]:
df

We see that there is one row for each model cell, and for each row, we see the distance, names, and weight for the "nearby" pilot points.  The interpolated value for cells that have a pilot point at their center only need one weight - 1.0 - and one pilot point.  Other cells are weighted combinations of pilot points.  Is this clear?  

Now we need to save the factors (weights) to a special file that we will use later to quickly generate a new HK array from a set of pilot point values:

In [ ]:
ok.to_grid_factors_file(pp_file+".fac")

In [ ]:
pp_file

Just for demo purposes, lets generate ``random`` pilot point values and run them through the factors to see what the ``hk`` array looks like

In [ ]:
# generate random values
df_pp.loc[:,"parval1"] = np.random.random(df_pp.shape[0])*25
# save a pilot points file
pyemu.utils.write_pp_file(pp_file,df_pp)

In [ ]:
# interpolate the pilot point values to the grid
hk_arr = pyemu.utils.fac2real(pp_file,factors_file=pp_file+".fac",out_file=None)

In [ ]:
# plot
ax = plt.subplot(111,aspect='equal')
cb=ax.imshow(hk_arr,interpolation="nearest",extent=m.sr.get_extent(),alpha=0.5)
ax.scatter(df_pp.x,df_pp.y,marker='.',s=4,color='k')
plt.colorbar(cb)